## Asignament Week 1

In [28]:
import pandas as pd
import numpy as np

In [29]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Asignament Week 3

In [30]:
import requests
import lxml.html as lh
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [32]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [33]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('{}:"{}"'.format(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


In [34]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=10:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [35]:
[len(C) for (title,C) in col]

[0, 0, 0]

In [36]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code\n,Borough\n,Neighbourhood\n


In [37]:
#Data 
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(URL)
print("There are : ",len(tables)," tables")
print("Take look at table 0")
print(tables[0])

There are :  3  tables
Take look at table 0
    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned 

In [38]:
toronto = tables[0]
toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [39]:
# Check that Postal Code is not repeteated
toronto['Postal Code'].unique

<bound method Series.unique of 0      M1A
1      M2A
2      M3A
3      M4A
4      M5A
      ... 
175    M5Z
176    M6Z
177    M7Z
178    M8Z
179    M9Z
Name: Postal Code, Length: 180, dtype: object>

### Cleaning the Dataframe 

In [40]:
#Delete al the cells with a borough that is 'Not assigned'
toronto.drop(toronto[toronto['Borough']=='Not assigned'].index, inplace = True )
toronto.shape

(103, 3)

In [41]:
#Check if a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough
toronto[toronto['Neighbourhood'] =='Not assigned']

,Postal Code,Borough,Neighbourhood


In [42]:
#Check again if theres repeteated Postal Codes
toronto['Postal Code'].value_counts()

M8W    1
M4V    1
M5K    1
M1M    1
M1N    1
      ..
M7A    1
M6S    1
M1T    1
M6K    1
M1G    1
Name: Postal Code, Length: 103, dtype: int64

In [43]:
#Proceed to read the CSV with the coordinates of each Postal Code 
mainpath = "/Users/Hugo/Downloads/"
filename = "Geospatial_Coordinates.csv"
data_coord = pd.read_csv(mainpath + "/" + filename,sep=",",header = 0)
data_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
#We merge the two dataframes to get the final result, with the coordinates
toronto_final = pd.merge(toronto, data_coord, on='Postal Code')

In [45]:
toronto_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [46]:
toronto_final.shape

(103, 5)